In [1]:
# from llmfact import LayerOutputExtractor, FBNFeatureExtractor, GroupFBNFeatureExtractor, FBNExtractor, LLMFC
# from llmfact.extractor import MutiLayerAnalysis
# from llmfact.extractor import SingleLayerAnalysis
# from llmfact.mask import MaskedGPT2ForSequenceClassification, MaskedGPT2AmplifiedForSequenceClassification, MaskedGPT2LMModel, MaskedModel
# from transformers import GPT2Model, GPT2Config, GPT2LMHeadModel, GPT2ForSequenceClassification, Trainer, TrainingArguments
# from transformers import GPT2Tokenizer
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForCausalLM
# from datasets import load_dataset, get_dataset_config_names, get_dataset_split_names
from torch.utils.data import DataLoader
# from rouge_score import rouge_scorer
# from evaluate import load

# from llmfact.utils import IoU, correlation_activation, thresholding, write_layer_txt, evaluate_iou
# from llmfact.stat import  StatICA, StatDictionaryLearning
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.decomposition import FastICA
import seaborn as sns
import pandas as pd
from tqdm.auto import tqdm
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3,4'
os.environ["TOKENIZERS_PARALLELISM"]  = "true"

In [2]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModel.from_pretrained("mistralai/Mistral-7B-v0.1", device_map="auto")

config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model

MistralModel(
  (embed_tokens): Embedding(32000, 4096)
  (layers): ModuleList(
    (0-31): 32 x MistralDecoderLayer(
      (self_attn): MistralAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
      )
      (mlp): MistralMLP(
        (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
        (act_fn): SiLUActivation()
      )
      (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
    )
  )
  (norm): MistralRMSNorm((4096,), eps=1e-05)
  (rotary_emb): MistralRotaryEmbedding()
)

In [5]:
include_layers = []
for i in range(32):
    include_layers.append(f'layers.{i}')
include_layers

['layers.0',
 'layers.1',
 'layers.2',
 'layers.3',
 'layers.4',
 'layers.5',
 'layers.6',
 'layers.7',
 'layers.8',
 'layers.9',
 'layers.10',
 'layers.11',
 'layers.12',
 'layers.13',
 'layers.14',
 'layers.15',
 'layers.16',
 'layers.17',
 'layers.18',
 'layers.19',
 'layers.20',
 'layers.21',
 'layers.22',
 'layers.23',
 'layers.24',
 'layers.25',
 'layers.26',
 'layers.27',
 'layers.28',
 'layers.29',
 'layers.30',
 'layers.31']

In [7]:
from datasets import load_dataset, get_dataset_config_names, get_dataset_split_names
wiki_dataset = load_dataset("Self-GRIT/wikitext-2-raw-v1-preprocessed", split='train')
inputs_list = [tokenizer(inputs, return_tensors="pt") for inputs in wiki_dataset['text'][:10]]

In [8]:
%%time
from llmfact.decomposition.canica import CanICA
from llmfact.utils import z_score_signals
from llmfact import LayerOutputExtractor
extractor = LayerOutputExtractor(model, include_layers, device=model.device)
layer_outputs_list = [extractor.extract_layer_outputs(inputs) for inputs in inputs_list]
# torch.save(layer_outputs_list, "./data/llama3-8b-wikitext2-10-4096.pth")
# torch.save(layer_outputs_list, "./data/llama2-7b-chat-hf-wikitext2-10-4096")
# torch.save(layer_outputs_list, "./data/chatglm2-6b-wikitext2-10-4096.pth")
torch.save(layer_outputs_list, "./data/mistral-7b-v0.1-wikitext2-10-4096.pth")

CPU times: user 34.5 s, sys: 5.26 s, total: 39.7 s
Wall time: 8.87 s


In [9]:
exit()